In [7]:
pip install pymc3 pandas numpy matplotlib

In [1]:
pip install --upgrade pymc numpy arviz

  Using cached numpy-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [3]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

data = pd.read_csv("Prices.csv")

# 1. esantion suficient de mare din distributia a posteriori
data['log_HardDrive'] = np.log(data['HardDrive']) #ln din marimea hard disk-ului

#extragem variabilele relevante pentru model
y = data['Price'].values  # y = pretul
x1 = data['Speed'].values  # x1 = frecventa procesorului MHz
x2 = data['log_HardDrive'].values  #x2 = ln din marimea hard disk-ului

#definim modelul bayesian y ∼ N(μ, σ), unde μ = α + β1 * x1 + β2 * x2
with pm.Model() as model: # modelul de regresie bayesiana
    #distributii a priori slab informative pentru parametrii
    alpha = pm.Normal("alpha", mu=0, sigma=100)  #intercept
    beta1 = pm.Normal("beta1", mu=0, sigma=10)  #coeficient pentru x1
    beta2 = pm.Normal("beta2", mu=0, sigma=10)  #coeficient pentru x2
    sigma = pm.HalfNormal("sigma", sigma=10)  #deviatia standard

    #relatia liniara
    mu = alpha + beta1 * x1 + beta2 * x2 #media distributiei normale pentru y (pretul)

    #observatia pentru variabila dependenta y
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

    #mostrare din distributia a posteriori
    trace = pm.sample(5000, tune=1000, random_seed=42)


# 2. calculam intervalele HDI de 95% pentru coeficientii β1 și β2
summary = az.summary(trace, hdi_prob=0.95) #rezumat al rezultatelor obtinute de distr a posteriori
print("HDI pentru coeficienti:")
print(summary.loc[['beta1', 'beta2'], ['mean', 'hdi_2.5%', 'hdi_97.5%']])

# 3. da, sunt; intervalele exclud 0, iar asta indica o relatie semnificativa intre frecventa procesorului / dimensiunea hard disk-ului si pretul

# 4. simularea pretului asteptat pentru PC-ul specificat
# frecventa 33 MHz, hard disk 540 MB
freq = 33
hd_size = 540
log_hd_size = np.log(hd_size)  # ln marimii hard diskului

#extragem mostrele parametrilor pentru parametrii alpha, beta1 si beta 2
alpha_samples = trace.posterior['alpha'].values.flatten()
beta1_samples = trace.posterior['beta1'].values.flatten()
beta2_samples = trace.posterior['beta2'].values.flatten()

#pretul așteptat (μ) pentru configuratia specificata
mu_samples = alpha_samples + beta1_samples * freq + beta2_samples * log_hd_size

# Intervalul HDI de 90% pentru prețul așteptat
mu_hdi = az.hdi(mu_samples, hdi_prob=0.90)
print(f"90% HDI pentru pretul asteptat: {mu_hdi}")


# 5. interval de predictie pentru pret
sigma_samples = trace.posterior['sigma'].values.flatten()  #mostre pentru deviatia standard
price_samples = np.random.normal(mu_samples, sigma_samples)  #preturi simulare
price_hdi = az.hdi(price_samples, hdi_prob=0.90)  #interval HDI
print(f"90% interval de predictie pentru pret: {price_hdi}")


# 6. afecteaza daca este premium?
premium_map = {'yes': 1, 'no': 0} #transf premium in val binara
data['PremiumBinary'] = data['Premium'].map(premium_map)
premium = data['PremiumBinary'].values #vector in care stocam valorile

#construim model care include feature-ul premium
with pm.Model() as premium_model:
    #distributii a priori
    alpha_p = pm.Normal("alpha", mu=0, sigma=100)
    beta_p = pm.Normal("beta", mu=0, sigma=10)  #coef pentru feature-ul premium
    sigma_p = pm.HalfNormal("sigma", sigma=10)

    #rel liniara incluzand premium
    mu_p = alpha_p + beta_p * premium #media distributiei normale pentru y (pretul)

    #observatia pentru variabila dependenta y
    y_obs_p = pm.Normal("y_obs", mu=mu_p, sigma=sigma_p, observed=y)

    #mostrare din distributia a posteriori
    premium_trace = pm.sample(2000, tune=1000, random_seed=42)

#rezumatul efectului Premium
premium_summary = az.summary(premium_trace, hdi_prob=0.95)
print("Efectul Premium (coeficient beta):")
print(premium_summary.loc['beta', ['mean', 'hdi_2.5%', 'hdi_97.5%']])
#intervalul HDI include 0, ceea ce inseamna ca premium nu afecteaza semnificativ statistic

Output()

Output()

HDI pentru coeficienti:
          mean  hdi_2.5%  hdi_97.5%
beta1    9.773     8.496     11.134
beta2  101.484    84.834    118.011
90% HDI pentru pretul asteptat: [2049.92638847 2115.38954094]
90% interval de predictie pentru pret: [1556.08389494 2572.44252914]


Output()

Output()

Efectul Premium (coeficient beta):
mean         14.164
hdi_2.5%     -4.509
hdi_97.5%    34.156
Name: beta, dtype: float64
